<a href="https://colab.research.google.com/github/prithuls/MultiviewSegmentation-CvAM/blob/main/Segmentation-CvAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os, glob, cv2
from skimage.io import imsave
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, \
              Conv2D, Add, MaxPooling2D, Activation, add, \
              Conv2DTranspose, BatchNormalization, Reshape, \
              GlobalAveragePooling2D, Multiply, Dense, \
              Permute, Multiply, Lambda, concatenate
from tensorflow.keras.optimizers import Adam, SGD, Adadelta, Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
#from data import load_train_data, load_test_data
from sklearn.model_selection import train_test_split
K.set_image_data_format('channels_last')
from scipy.spatial.distance import directed_hausdorff

In [ ]:
## Configuration

train_full_dir = "/content/gdrive/MyDrive/CS791MassDetectionOnMammograms/Data(CBIS-DDSM)/processed/mass-train-full"
train_mask_combined_dir = "/content/gdrive/MyDrive/CS791MassDetectionOnMammograms/Data(CBIS-DDSM)/processed/mass-train-mask-combined"
test_full_dir = "/content/gdrive/MyDrive/CS791MassDetectionOnMammograms/Data(CBIS-DDSM)/processed/mass-test-full"
test_mask_combined_dir = "/content/gdrive/MyDrive/CS791MassDetectionOnMammograms/Data(CBIS-DDSM)/processed/mass-test-mask-combined"
extension = ".png"

target_size = 224

brightness_delta = 0.3

batch_size = 8
smooth = 1.

encoder_input_shape = (target_size, target_size, 3)
decoder_kernel_size = (3, 3)
decoder_strides = (2, 2)
decoder_padding = 'same'
decoder_activation = None
final_layer_filters = 1
final_layer_activation = "sigmoid"
learning_rate = 0.0001

## **Preprocessing of Multi-View Images**

In [ ]:
def datasetPaths(full_img_dir, mask_img_dir, extension):

    """
    Follows the flow:
    1.         datasetPaths() *
              _______|_______
            |               |
    2. loadFullImg()  loadMaskImg()
            |_______________|
                    |
    3.          tfParse()
                    |
    4.         imgAugment()
                    |
    5.        makeTFDataset()
    Takes in the directories of the folder containing the full mammogram
    scans (x) and the ground truth masks (y) and returns a list of paths to
    individual full scans and masks.
    Parameters
    ----------
    full_img_dir : {str}
        Directory that contains the FULL training images.
    mask_img_dir : {str}
        Directory that contains the MASK training images.
    extension : {str}
        The file extension of the images (e.g. ".png")
    Returns
    -------
    x_paths_list: {list}
        A list of paths to individual FULL images in the training set.
    y_paths_list: {list}
        A list of paths to individual MASK images in the training set.
    """

    try:

        # =======================================
        #  1. Get paths of X (full) and y (mask)
        # =======================================

        x_paths_list = []
        y_paths_list = []

        full_img_ID = [fn[:26] for fn in sorted(os.listdir(full_img_dir))]
        mask_img_ID = [fn[:26] for fn in sorted(os.listdir(mask_img_dir))]

        full_dirs = [fn for fn in sorted(os.listdir(full_img_dir))]
        mask_dirs = [fn for fn in sorted(os.listdir(mask_img_dir))]

        ## Common Elements
        common_ID_FullandMask = []
        temp_mask_img_ID = mask_img_ID
        for a_ in full_img_ID:
          for b_ in temp_mask_img_ID:
            if a_ == b_:
              common_ID_FullandMask.append(a_)
              temp_mask_img_ID.remove(b_)

        temp_full_path, temp_mask_path = 0, 0 ## For checking if the 'Below function ifs' are picking anything

        for i in range(len(common_ID_FullandMask)):
            for j in range(len(full_dirs)):
                fn_ = full_dirs[j]
                if common_ID_FullandMask[i] in fn_ and fn_.endswith(extension):
                    temp_full_path = fn_
                    break
            for k in range(len(mask_dirs)):
                fn_2 = mask_dirs[k]
                if common_ID_FullandMask[i] in fn_2 and fn_2.endswith(extension) :
                    temp_mask_path = fn_2
                    break
            if temp_full_path == 0 or temp_mask_path == 0:
                temp_full_path, temp_mask_path = 0, 0
                continue
            
            full_dirs.pop(j)
            mask_dirs.pop(k)
            x_paths_list.append(os.path.join(full_img_dir, temp_full_path))
            y_paths_list.append(os.path.join(mask_img_dir, temp_mask_path))

            temp_full_path, temp_mask_path = 0, 0 # Reasserting value to 0


        print('x_paths_list length', len(x_paths_list))
        print('y_paths_list length', len(y_paths_list))


        # Get paths of train images and masks.
        '''for full in os.listdir(full_img_dir):
            if full.endswith(extension):
                x_paths_list.append(os.path.join(full_img_dir, full))

        for mask in os.listdir(mask_img_dir):
            if mask.endswith(extension):
                y_paths_list.append(os.path.join(mask_img_dir, mask))'''

        # ** IMPORTANT ** Sort so that FULL and MASK images are in an order
        # that corresponds with each other.
        #x_paths_list.sort()
        #y_paths_list.sort()

    except Exception as e:
        # logger.error(f'Unable to datasetPaths!\n{e}')
        print((f"Unable to datasetPaths!\n{e}"))

    return x_paths_list, y_paths_list


def create_dict(full_dir, mask_combined_dir):
    full_img_files = sorted(os.listdir(full_dir))
    mask_img_files = sorted(os.listdir(mask_combined_dir))

    if "train" in full_dir:
        str_start, str_end = 16, 27
    elif "test" in full_dir:
        str_start, str_end = 12, 23

    hm_files = {}
    for f1, f2 in zip(full_img_files, mask_img_files):
        temp_key = f1[str_start : str_end]    ## Key is the patient id and left/right breast
        if temp_key not in hm_files:
            hm_files[temp_key] = [[], []]
        hm_files[temp_key][0].append(os.path.join(full_dir, f1))
        hm_files[temp_key][1].append(os.path.join(mask_combined_dir, f2))

    # for key in hm_files:
    #     print(key, hm_files[key])

    count = 0
    keys_to_pop = []
    for key in hm_files:
        if len(hm_files[key][0]) == 1:
            keys_to_pop.append(key)
            count += 1

    print("Before popping, number of keys to pop with length 1: ", count, " and total length: ", len(hm_files.keys()))

    for key in keys_to_pop:
        hm_files.pop(key, None)

    ## To check if the dictionary has any key with length 1
    count = 0
    for key in hm_files:
        if len(hm_files[key][0]) == 1:
            count += 1
            # print(key)

    print("After popping, number of keys to pop with length 1: ", count)
    print("After popping, number of keys with length 2: ", len(hm_files.keys()))

    return hm_files

def loadFullImg(path, dsize):

    """
    Takes in the path of a FULL image and loads it as a grayscale image (as
    a numpy array with 3 channels).
    Parameters
    ----------
    path : {str}
        The path of the FULL image.
    Returns
    -------
    full_img : {numpy.ndarray}
        The loaded image with shape = (self.target_size, self.target_size, 3)
    """

    try:
        # =====================================
        #  2a. Read images (full)
        # =====================================
        # if not isinstance(path, str):
        #     path = path.decode()

        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(src=img, dsize=(dsize, dsize))

        # Min max normalise to [0, 1].
        norm_img = (img - img.min()) / (img.max() - img.min())

        # Stack grayscale image to make channels=3.
        full_img = np.stack([norm_img, norm_img, norm_img], axis=-1)

    except Exception as e:
        # logger.error(f'Unable to loadFullImg!\n{e}')
        print((f"Unable to loadFullImg!\n{e}"))

    return full_img

def loadMaskImg(path, dsize):

    """
    Takes in the path of a MASK image and loads it as a grayscale image (as
    a numpy array with 1 channel).
    Parameters
    ----------
    path : {str}
        The path of the MASK image.
    Returns
    -------
    full_img : {numpy.ndarray}
        The loaded image with shape = (self.target_size, self.target_size, 1)
    """

    try:

        # ========================
        #  2b. Read images (mask)
        # ========================
        # if not isinstance(path, str):
        #     path = path.decode()

        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(src=img, dsize=(dsize,dsize))

        # Min max normalise to [0, 1].
        norm_img = (img - img.min()) / (img.max() - img.min())

        # Expand shape to (width, height, 1).
        mask_img = np.expand_dims(norm_img, axis=-1)

    except Exception as e:
        # logger.error(f'Unable to loadMaskImg!\n{e}')
        print((f"Unable to loadMaskImg!\n{e}"))

    return mask_img

def tfParse(img_cc_path, img_mlo_path, mask_cc_path, mask_mlo_path):

    """
    Takes in a FULL image path and a MASK image path, loads the images using
    loadFullImg() and loadMaskImg() and makes it TensorFlow executable.
    Parameters
    ----------
    x_path: {str}
        The path of the FULL image to be loaded.
    y_path: {str}
        The path of the MASK image to be loaded.
    Returns
    -------
    x : {tensor}
        The FULL image loaded as a tensor with shape =
        (self.target_size, self.target_size, 3)
    y : {tensor}
        The MASK image loaded as a tensor with shape =
        (self.target_size, self.target_size, 1)
    """

    def _parse(x_path, y_path):
        x = loadFullImg(path=x_path, dsize=target_size)
        y = loadMaskImg(path=y_path, dsize=target_size)
        return x, y

    try:
        # ===========
        #  3. Parse
        # ===========

        # x_cc, y_cc = tf.numpy_function(_parse, [img_cc_path, mask_cc_path], [tf.float64, tf.float64])
        # x_mlo, y_mlo = tf.numpy_function(_parse, [img_mlo_path, mask_mlo_path], [tf.float64, tf.float64])

        x_cc, y_cc = _parse(img_cc_path, mask_cc_path)
        x_mlo, y_mlo = _parse(img_mlo_path, mask_mlo_path)

        print(np.shape(x_cc), np.shape(y_cc))

        # x_cc.set_shape([target_size, target_size, 3])
        # x_mlo.set_shape([target_size, target_size, 3])
        # y_cc.set_shape([target_size, target_size, 1])
        # y_mlo.set_shape([target_size, target_size, 1])

        x = concatenate([x_cc, x_mlo], axis = -1)
        y = concatenate([y_cc, y_mlo], axis = -1)

        return x, y

    except Exception as e:
        # logger.error(f'Unable to tfParse!\n{e}')
        print((f"Unable to tfParse!\n{e}"))

def imgAugment(x_img, y_img):

    """
    Apply random image augmentation to full mammogram scans (x_img). Note
    that the same augmentation has to be applied to the masks (y_img), apart
    from the brightness augmentation.
    Parameters
    ----------
    x_img : {numpy.ndarray}
        Full mammogram scan to augment.
    y_img : {numpy.ndarray}
        Corresponding mask of `x_img`.
    Returns
    -------
    x_img : {numpy.ndarray}
        Augmented x_img.
    y_img : {numpy.ndarray}
        Augmented y_img.
    """

    try:

        # =========
        #  LR Flip
        # =========
        # Generate random number from uniform distribution
        # in the range [0.0, 1.0)
        if tf.random.uniform(()) > 0.5:
            x_img = tf.image.flip_left_right(image=x_img)
            y_img = tf.image.flip_left_right(image=y_img)

        # =========
        #  UD Flip
        # =========
        if tf.random.uniform(()) > 0.5:
            x_img = tf.image.flip_up_down(image=x_img)
            y_img = tf.image.flip_up_down(image=y_img)

        # ============
        #  Brightness
        # ============
        # Only change the brightness of x_img, not y_img!
        x_img = tf.image.random_brightness(
            image=x_img, max_delta= brightness_delta
        )

    except Exception as e:
        # logger.error(f'Unable to imgAugment!\n{e}')
        print((f"Unable to imgAugment!\n{e}"))

    return x_img, y_img

def makeTFDataset(shuffle, augment, x_paths_list, y_paths_list, batch_size):

    """
    Takes in a FULL image path and a MASK image path, loads the images using
    loadFullImg() and loadMaskImg() and makes it TensorFlow executable.
    Parameters
    ----------
    shuffle : {boolean}
        If True, shuffle dataset. Should only shuffle train set.
    augment : {boolean}
        If True, perform augmentation. Should only augment test set.
    x_paths_list : {list}
        The path of the FULL image to be loaded.
    y_paths_list : {list}
        The path of the MASK image to be loaded.
    batch_size : {int}
        The batch size to create the dataset with.
    Returns
    -------
    ds : {RepeatDataset}
        The dataset with x (FULL) and y (MASK) images loaded from
        `x_paths_list` and `y_paths_list`.
        For each `batch` in `ds`:
        - type = tuple
        - len = 2
        - `batch[0]` (this is a batch of the FULL images):
            - type = tensorflow.python.fraework.ops.EagerTensor
            - shape = (self.batch_size, self.target_size, self.target_size, 3)
            - `batch[0][0]` (this is the first FULL image in the batch):
                - type = tensorflow.python.fraework.ops.EagerTensor
                - shape = (self.target_size, self.target_size, 3)
        - `batch[1]` (this is a batch the MASK images):
            - type = tensorflow.python.fraework.ops.EagerTensor
            - shape = (self.batch_size, self.target_size, self.target_size, 1)
                - `batch[1][0]` (this is the first MASK image in the batch):
                - type = tensorflow.python.fraework.ops.EagerTensor
                - shape = (self.target_size, self.target_size, 1)
    """

    try:
        # ====================
        #  4. Make TF Dataset
        # ====================

        ds = tf.data.Dataset.from_tensor_slices((x_paths_list, y_paths_list))

        # Shuffle the paths of the elements
        # with buffer_size=len(x_paths_list), since storing all the paths
        # in memory will not be an issue (as compared to storing all the
        # imported images as tensors).
        if shuffle:
            ds = ds.shuffle(buffer_size=len(x_paths_list))

        # Transform paths to images after shuffling the paths.
        ds = ds.map(tfParse, num_parallel_calls=tf.data.experimental.AUTOTUNE)

        # Apply image augmentation.
        if augment:
            ds = ds.map(
                imgAugment, num_parallel_calls=tf.data.experimental.AUTOTUNE
            )

        # Batch only AFTER shuffle, so that we shuffled the elements not the batches.
        ds = ds.batch(batch_size=batch_size, drop_remainder=False)
        ds = ds.repeat()  # We need to repeat in order to train for > 1 epoch.

    except Exception as e:
        # logger.error(f'Unable to makeTFDataset!\n{e}')
        print((f"Unable to makeTFDataset!\n{e}"))

    return ds

In [ ]:
hm_train_files_multi_view = create_dict(train_full_dir, train_mask_combined_dir)
hm_test_files_multi_view = create_dict(test_full_dir, test_mask_combined_dir)

Before popping, number of keys to pop with length 1:  213  and total length:  722
After popping, number of keys to pop with length 1:  0
After popping, number of keys with length 2:  509
Before popping, number of keys to pop with length 1:  59  and total length:  211
After popping, number of keys to pop with length 1:  0
After popping, number of keys with length 2:  152


In [ ]:
X_train, y_train = [], []

for key in hm_train_files_multi_view:
    img_cc_path, img_mlo_path = sorted(hm_train_files_multi_view[key][0])
    mask_cc_path, mask_mlo_path = sorted(hm_train_files_multi_view[key][1])

    x, y = tfParse(img_cc_path, img_mlo_path, mask_cc_path, mask_mlo_path)
    X_train.append(x)
    y_train.append(y)

In [ ]:
X_test, y_test = [], []

for key in hm_test_files_multi_view:
    img_cc_path, img_mlo_path = sorted(hm_test_files_multi_view[key][0])
    mask_cc_path, mask_mlo_path = sorted(hm_test_files_multi_view[key][1])

    x, y = tfParse(img_cc_path, img_mlo_path, mask_cc_path, mask_mlo_path)
    X_test.append(x)
    y_test.append(y)

In [ ]:
input_cc = X_test[0][ :, :, : 3]
input_mlo = X_test[0][ :, :, 3 :]

In [ ]:
np.shape(input_cc)

TensorShape([224, 224, 3])

In [ ]:
for key in hm_test_files_multi_view:
    img_cc_path, img_mlo_path = sorted(hm_test_files_multi_view[key][0])
    mask_cc_path, mask_mlo_path = sorted(hm_test_files_multi_view[key][1])

    x, y = tfParse(img_cc_path, img_mlo_path, mask_cc_path, mask_mlo_path)
    break

(224, 224, 3) (224, 224, 1)


In [ ]:
np.shape(x[:, :, : 3])


TensorShape([224, 224, 3])

In [ ]:
np.shape(y_train[1])

TensorShape([224, 224, 2])

## **Metrics**

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    
def dice(pred, true, k = 1):
    intersection = np.sum(pred[true==k]) * 2.0
    dice = intersection / (np.sum(pred) + np.sum(true))
    return dice

def iou(pred, true, k = 1):
    intersection = np.sum(pred[true==k])
    iou = intersection / (np.sum(pred) + np.sum(true) - intersection)
    return iou

def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou

def jacard(y_true, y_pred):

    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum ( y_true_f * y_pred_f)
    union = K.sum ( y_true_f + y_pred_f - y_true_f * y_pred_f)

    return intersection/union

def sens(y_true, y_pred):
    num=K.sum(K.multiply(y_true, y_pred))
    denom=K.sum(y_true)
    if denom==0:
        return 1
    else:
        return  num/denom

def spec(y_true, y_pred):
    num=K.sum(K.multiply(y_true==0, y_pred==0))
    denom=K.sum(y_true==0)
    if denom==0:
        return 1
    else:
        return  num/denom

def tversky(y_true, y_pred):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.7
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

def focal_tversky(y_true,y_pred):
    pt_1 = tversky(y_true, y_pred)
    gamma = 0.75
    return K.pow((1-pt_1), gamma)


## From TDS guy

def iouMetric(y_true, y_pred):

      """
      Calculates the intersection-over-union between a ground truth mask and
      the corresponding predicted mask from the model.
      Parameters
      ----------
      y_true : {tf.Tensor}
          The ground truth mask.
      y_pred : {tf.Tensor}
          Corresponding predicted mask.
      Returns
      -------
      compute_iou() : {tf.numpy_function}
          A function that calculates the IOU metric.
      """

      try:

          def compute_iou(y_true, y_pred):
              intersection = (y_true * y_pred).sum()
              union = y_true.sum() + y_pred.sum() - intersection
              x = (intersection + 1e-15) / (union + 1e-15)
              x = x.astype(np.float32)
              return x

          return tf.numpy_function(compute_iou, [y_true, y_pred], tf.float32)

      except Exception as e:
          # logger.error(f'Unable to iouMetric!\n{e}')
          print((f"Unable to iouMetric!\n{e}"))

## **CvAM with Basic U-Net**

In [ ]:
def cvam_spatial_attention(x_cc,x_mlo):
    avg_pool_cc = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(x_cc)
    assert avg_pool_cc.shape[-1] == 1
    max_pool_cc = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(x_cc)
    assert max_pool_cc.shape[-1] == 1

    avg_pool_mlo = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(x_mlo)
    assert avg_pool_mlo.shape[-1] == 1
    max_pool_mlo = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(x_mlo)
    assert max_pool_mlo.shape[-1] == 1

    concat = concatenate(axis=3)([avg_pool_cc, max_pool_cc, avg_pool_mlo, max_pool_mlo])
    assert concat.shape[-1] == 4
    cvam_feature = Conv2D(4, (3, 3), padding="same")(concat)
    cvam_feature = Activation('relu')(cvam_feature)
    cvam_feature = Conv2D(1, (3, 3), padding="same")(cvam_feature)
    cvam_feature = Activation('sigmoid')(cvam_feature)
    #print (cvam_feature)
    assert cvam_feature.shape[-1] == 1
    # if K.image_data_format() == "channels_first":
    #   cvam_feature = Permute((3, 1, 2))(cvam_feature)

    # print (multiply([x, cvam_feature]))
    # return multiply([x, cvam_feature])

    print(Multiply([x_cc, cvam_feature]))
    print(Multiply([x_mlo, cvam_feature]))

    cvam_cc, cvam_mlo = Multiply([x_cc, cvam_feature]), Multiply([x_mlo, cvam_feature])
    return cvam_cc, cvam_mlo

In [ ]:
def unet_with_cvam():
    inputs = Input((target_size, target_size, 6))

    ## Splitting the inputs into cc and mlo

    input_cc = inputs[:, :, :, : 3]
    input_mlo = inputs[:, :, :, 3 :]

    ## 1st Layer

    conv1_cc = Conv2D(32, (3, 3), activation='relu', padding='same')(input_cc)
    conv1_cc = BatchNormalization()(conv1_cc)
    conv1_cc = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1_cc)
    conv1_cc = BatchNormalization()(conv1_cc)
    pool1_cc = MaxPooling2D((2,2), padding='same')(conv1_cc)

    print(np.shape(pool1_cc))

    conv1_mlo = Conv2D(32, (3, 3), activation='relu', padding='same')(input_mlo)
    conv1_mlo = BatchNormalization()(conv1_mlo)
    conv1_mlo = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1_mlo)
    conv1_mlo = BatchNormalization()(conv1_mlo)
    pool1_mlo = MaxPooling2D((2,2), padding='same')(conv1_mlo)

    ## 2nd Layer

    conv2_cc = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1_cc)
    conv2_cc = BatchNormalization()(conv2_cc)
    conv2_cc = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2_cc)
    conv2_cc = BatchNormalization()(conv2_cc)
    pool2_cc = MaxPooling2D((2,2), padding='same')(conv2_cc)

    print(np.shape(pool2_cc))

    conv2_mlo = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1_mlo)
    conv2_mlo = BatchNormalization()(conv2_mlo)
    conv2_mlo = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2_mlo)
    conv2_mlo = BatchNormalization()(conv2_mlo)
    pool2_mlo = MaxPooling2D((2,2), padding='same')(conv2_mlo)

    ## 3rd Layer

    conv3_cc = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2_cc)
    conv3_cc = BatchNormalization()(conv3_cc)
    conv3_cc = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3_cc)
    conv3_cc = BatchNormalization()(conv3_cc)
    pool3_cc = MaxPooling2D((2,2), padding='same')(conv3_cc)

    conv3_mlo = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2_mlo)
    conv3_mlo = BatchNormalization()(conv3_mlo)
    conv3_mlo = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3_mlo)
    conv3_mlo = BatchNormalization()(conv3_mlo)
    pool3_mlo = MaxPooling2D((2,2), padding='same')(conv3_mlo)


    ## 4th Layer

    conv4_cc = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3_cc)
    conv4_cc = BatchNormalization()(conv4_cc)
    conv4_cc = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4_cc)
    conv4_cc = BatchNormalization()(conv4_cc)
    pool4_cc = MaxPooling2D((2,2), padding='same')(conv4_cc)

    conv4_mlo = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3_mlo)
    conv4_mlo = BatchNormalization()(conv4_mlo)
    conv4_mlo = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4_mlo)
    conv4_mlo = BatchNormalization()(conv4_mlo)
    pool4_mlo = MaxPooling2D((2,2), padding='same')(conv4_mlo)

    ## 5th Layer

    conv5_cc = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4_cc)
    conv5_cc = BatchNormalization()(conv5_cc)
    conv5_cc = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5_cc)
    conv5_cc = BatchNormalization()(conv5_cc)

    conv5_mlo = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4_mlo)
    conv5_mlo = BatchNormalization()(conv5_mlo)
    conv5_mlo = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5_mlo)
    conv5_mlo = BatchNormalization()(conv5_mlo)

    ## 6th Layer

    up6_cc = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5_cc), conv4_cc], axis=3)
    conv6_cc = Conv2D(256, (3, 3), activation='relu', padding='same')(up6_cc)
    conv6_cc = BatchNormalization()(conv6_cc)
    conv6_cc = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6_cc)
    conv6_cc = BatchNormalization()(conv6_cc)

    up6_mlo = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5_mlo), conv4_mlo], axis=3)
    conv6_mlo = Conv2D(256, (3, 3), activation='relu', padding='same')(up6_mlo)
    conv6_mlo = BatchNormalization()(conv6_mlo)
    conv6_mlo = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6_mlo)
    conv6_mlo = BatchNormalization()(conv6_mlo)

    ## 7th Layer
    
    up7_cc = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6_cc), conv3_cc], axis=3)
    conv7_cc = Conv2D(128, (3, 3), activation='relu', padding='same')(up7_cc)
    conv7_cc = BatchNormalization()(conv7_cc)
    conv7_cc = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7_cc)
    conv7_cc = BatchNormalization()(conv7_cc)

    up7_mlo = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6_mlo), conv3_mlo], axis=3)
    conv7_mlo = Conv2D(128, (3, 3), activation='relu', padding='same')(up7_mlo)
    conv7_mlo = BatchNormalization()(conv7_mlo)
    conv7_mlo = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7_mlo)
    conv7_mlo = BatchNormalization()(conv7_mlo)

    ## 8th Layer

    up8_cc = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7_cc), conv2_cc], axis=3)
    conv8_cc = Conv2D(64, (3, 3), activation='relu', padding='same')(up8_cc)
    conv8_cc = BatchNormalization()(conv8_cc)
    conv8_cc = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8_cc)
    conv8_cc = BatchNormalization()(conv8_cc)

    up8_mlo = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7_mlo), conv2_mlo], axis=3)
    conv8_mlo = Conv2D(64, (3, 3), activation='relu', padding='same')(up8_mlo)
    conv8_mlo = BatchNormalization()(conv8_mlo)
    conv8_mlo = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8_mlo)
    conv8_mlo = BatchNormalization()(conv8_mlo)

    ## 9th Layer
    
    up9_cc = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8_cc), conv1_cc], axis=3)
    conv9_cc = Conv2D(32, (3, 3), activation='relu', padding='same')(up9_cc)
    conv9_cc = BatchNormalization()(conv9_cc)
    conv9_cc = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9_cc)
    conv9_cc = BatchNormalization()(conv9_cc)

    up9_mlo = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8_mlo), conv1_mlo], axis=3)
    conv9_mlo = Conv2D(32, (3, 3), activation='relu', padding='same')(up9_mlo)
    conv9_mlo = BatchNormalization()(conv9_mlo)
    conv9_mlo = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9_mlo)
    conv9_mlo = BatchNormalization()(conv9_mlo)

    ## Final Layer
        
    conv10_cc = Conv2D(1, (1, 1), activation='sigmoid')(conv9_cc)
    conv10_mlo = Conv2D(1, (1, 1), activation='sigmoid')(conv9_mlo)

    ## Stacking the final masks on top of another
    conv10 = concatenate([conv10_cc, conv10_mlo], axis= -1)


    model = Model(inputs=[inputs], outputs=[conv10])
    model.compile(optimizer = Adam(1e-3), loss = 'binary_crossentropy', metrics = [dice_coef, jacard, 'accuracy'])
    model.summary()
    return model

In [ ]:
unet_cvam = unet_with_cvam()

In [ ]:
history = unet_cvam.fit(X_train, y_train,
                    validation_split = 0.2,
                    epochs=300,
                    batch_size = 8,
                    # steps_per_epoch=train_steps,
                    # validation_steps=val_steps,
                    # callbacks=[model_checkpoint, reduce_lr, early_S],
                    verbose=1,
                )   

Epoch 1/300


ValueError: ignored

In [ ]:
train_steps = train_size // batch_size
val_steps =  val_size // batch_size
test_steps = test_size // batch_size

if train_size % batch_size != 0:
    train_steps += 1
if val_size % batch_size != 0:
    val_steps += 1
if test_size % batch_size != 0:
    test_steps += 1



#loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
#optimizer = tf.keras.optimizers.Adam(learning_rate= learning_rate)
#metrics=[dice_coef, jacard, iou_coef, 'accuracy']
#loss = modified_loss()
#u_net_inbreast.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=metrics)

fname = "u_net_inbreast_epoch_{epoch:02d}_-dice_{val_dice_coef:.2f}-iou_{val_jacard:.2f}+395.hdf5"
model_checkpoint = ModelCheckpoint(fname, monitor='val_loss', save_best_only=False)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
early_S = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=40)

print('-'*30)
print('Fitting u_net_inbreast...')
print('-'*30)

history = u_net_inbreast.fit(train_ds,
                    validation_data=val_ds,
                    epochs=300,
                    steps_per_epoch=train_steps,
                    validation_steps=val_steps,
                    callbacks=[model_checkpoint, reduce_lr, early_S],
                    verbose=1,
                )   